In [1]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import os
from nilearn.image import load_img
import pandas as pd
import gc
from collections import OrderedDict

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

data_dir = 'data'
pet_dir = "data/ad_pet_huw"
os.makedirs('data/pet_csf', exist_ok=True)

In [2]:
files = {}
for file in os.listdir(pet_dir):
    if file.endswith(".nii"):
        img = load_img(os.path.join(pet_dir, file))
        patient_id = file.split(".")[0].removeprefix('AD_normalised_')
        print(f'Processing patient: {patient_id}')
        # Convert the image to a PyTorch tensor
        torch_img = torch.tensor(img.get_fdata(), dtype=torch.float32)
        files[patient_id] = torch_img

Processing patient: 002_S_5018
Processing patient: 003_S_4136
Processing patient: 003_S_4152
Processing patient: 003_S_4373
Processing patient: 003_S_4892
Processing patient: 003_S_5165
Processing patient: 003_S_5187
Processing patient: 005_S_4707
Processing patient: 005_S_4910
Processing patient: 005_S_5038
Processing patient: 005_S_5119
Processing patient: 006_S_4153
Processing patient: 006_S_4192
Processing patient: 006_S_4546
Processing patient: 006_S_4867
Processing patient: 007_S_4568
Processing patient: 007_S_4637
Processing patient: 007_S_4911
Processing patient: 007_S_5196
Processing patient: 009_S_5027
Processing patient: 009_S_5037
Processing patient: 009_S_5224
Processing patient: 009_S_5252
Processing patient: 011_S_4827
Processing patient: 011_S_4845
Processing patient: 011_S_4906
Processing patient: 011_S_4912
Processing patient: 011_S_4949
Processing patient: 013_S_5071
Processing patient: 014_S_4039
Processing patient: 014_S_4615
Processing patient: 016_S_4009
Processi

In [3]:
df = pd.read_csv(os.path.join(data_dir, 'ADNIMERGE_19Jun2025.csv'))
df['ABETA_bl'] = df['ABETA_bl'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
df['PTAU_bl'] = df['PTAU_bl'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
df['TAU_bl'] = df['TAU_bl'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
df['tau_ab_ratio'] = df['TAU_bl'] / df['ABETA_bl']
df['ptau_ab_ratio'] = df['PTAU_bl'] / df['ABETA_bl']
df['A+'] = df['ABETA_bl'].apply(lambda x: 1 if x < 880 else 0)
df['T+'] = df['ptau_ab_ratio'].apply(lambda x: 1 if x > 0.028 else 0)
df['N+'] = df['tau_ab_ratio'].apply(lambda x: 1 if x > 0.33 else 0)
df = df.filter(['PTID', 'A+', 'T+', 'N+'])
df.head()

C:\Users\Dhruv Khanna\AppData\Local\Temp\ipykernel_19596\4162410189.py:1: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(data_dir, 'ADNIMERGE_19Jun2025.csv'))


,PTID,A+,T+,N+
0,011_S_0002,0,0,0
1,011_S_0003,1,1,0
2,011_S_0003,1,1,0
3,011_S_0003,1,1,0
4,011_S_0003,1,1,0


In [4]:
# Compute the number of common patients between the PET files and the df
missing_patients = df[~df['PTID'].isin(files.keys())]
print(f'Missing patients: {len(missing_patients)}')

common_patients = df['PTID'].isin(files.keys())
print(f'Common patients: {common_patients.sum()}')

print(f'Total patients {len(files)}')

Missing patients: 15684
Common patients: 737
Total patients 149


In [5]:
# Update the DataFrame to include a new column for the PET image data matched on PTID
for patient in files:
    img = files.get(patient)
    df['PET_IMAGE'] = df['PTID'].map(files) # Insert the img data ino the 'PET_IMAGE' column in df for the corresponding PTID field

In [6]:
# Print the PTID for the columns for which PET_IMAGE is not None
print(f'Number of patients: {len(df)}')
df.dropna(subset=['PET_IMAGE'], inplace=True)
df.drop_duplicates(subset=['PTID'], inplace=True)
print(f'Number of patients with PET images: {len(df)}')

Number of patients: 16421
Number of patients with PET images: 149


In [7]:
class PETDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image.unsqueeze(0), label  # Add channel dimension for CNN input

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
torch.cuda.empty_cache()
print(f'Using device: {device}')

Using device: cuda


In [8]:
class DeepPETModel(torch.nn.Module):
    def __init__(self):
        super(DeepPETModel, self).__init__()
        pass

    def inheritance_test(self):
        print("DeepPETModel inheritance test passed!")

    def input_gradient_hook(self, gradients):
        """
        cature gradient with respect to input
        """

        # print("triggered input gradient hook")
        self.input_gradient = gradients

    def activation_gradient_hook(self, gradients):
        """
        capture gradient with respect to activation maps
        """

        # print("triggered activation gradient hook")
        self.activation_gradients = gradients

    def get_activation_maps(self, x):
        """
        return the activation maps of the last convolutional block
        """

        return self.activation_maps

    def get_input_gradient(self):
        """
        retrieve gradient with respect to input
        """

        return self.input_gradient

    def get_activation_gradients(self):
        """
        retrieve gradient with respect to activation maps
        """

        return self.activation_gradients


class PreActivationResBlock(torch.nn.Module):
    def __init__(self, planes):
        super().__init__()
        self.conv1 = torch.nn.Conv3d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.conv2 = torch.nn.Conv3d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn1 = torch.nn.BatchNorm3d(planes)
        self.bn2 = torch.nn.BatchNorm3d(planes)
        self.relu = torch.nn.ReLU(inplace=True)
        self.dropout = torch.nn.Dropout3d(p=0.25)

    def preresidual_gradient_hook(self, gradients):
        """
        cature gradient with respect to input
        """
        self.preresidual_gradients = gradients

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.dropout(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)
        if (not self.training) and (x.requires_grad):
            # print(f"PreActivationResBlock: triggered gradient hook")
            h = x.register_hook(self.preresidual_gradient_hook)
        self.preresidual_activation_maps = out.detach().clone()
        out += x

        return out


class DeepPETEncoder(DeepPETModel):
    def __init__(self):
        super().__init__()

        self.layer0 = self._make_layer(1, 8, stride=1)
        self.layer1 = self._make_layer(8, 16, stride=2)
        self.layer2 = self._make_layer(16, 32, stride=2)
        self.layer3 = self._make_layer(32, 64, stride=2)
        self.layer4 = self._make_layer(64, 128, stride=2)

        self.output = torch.nn.Sequential(
            torch.nn.Dropout(p=0.50),
            torch.nn.Linear(128, 1),
        )

        self.layers = [self.layer0, self.layer1, self.layer2, self.layer3, self.layer4]

        self.input_gradient = None
        self.gradients = None

    def _make_layer(self, in_planes, out_planes, stride=1):

        layers = [
            torch.nn.Conv3d(
                in_planes,
                out_planes,
                kernel_size=3,
                stride=stride,
                padding=1,
                bias=False,
            )
        ]
        layers.append(PreActivationResBlock(planes=out_planes))

        return torch.nn.Sequential(*layers)

    def forward(self, x):

        if (not self.training) and (x.requires_grad):
            h = x.register_hook(self.input_gradient_hook)

        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        if (not self.training) and (x.requires_grad):
            # print(f"DeepPETEncoder: triggered gradient hook")
            h = x.register_hook(self.activation_gradient_hook)

        # global average pooling 3d
        x = x.mean(dim=(-3, -2, -1))
        x = self.output(x)

        return x


class PreActivationResBlockGradCAM(torch.nn.Module):
    """
    GradCAM-compatible PreActivaitonResBlock
    """

    def __init__(self, planes):
        super().__init__()
        self.conv1 = torch.nn.Conv3d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.conv2 = torch.nn.Conv3d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn1 = torch.nn.BatchNorm3d(planes)
        self.bn2 = torch.nn.BatchNorm3d(planes)
        self.relu = torch.nn.ReLU(inplace=True)
        self.dropout = torch.nn.Dropout3d(p=0.25)

    def preresidual_gradient_hook(self, gradients):
        """
        cature gradient with respect to input
        """
        self.preresidual_gradients = gradients

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.dropout(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)
        if (not self.training) and (x.requires_grad):
            # print(f"PreActivationResBlock: triggered gradient hook")
            h = x.register_hook(self.preresidual_gradient_hook)
        self.preresidual_activation_maps = out.detach().clone()
        out += x

        return out


class DeepPETEncoderGradCAM(DeepPETModel):
    """
    GradCAM-compatible DeepPETEncoder
    """

    def __init__(self):
        super().__init__()

        self.conv_layer0 = self._make_conv_layer(1, 8, stride=1)
        self.conv_layer1 = self._make_conv_layer(8, 16, stride=2)
        self.conv_layer2 = self._make_conv_layer(16, 32, stride=2)
        self.conv_layer3 = self._make_conv_layer(32, 64, stride=2)
        self.conv_layer4 = self._make_conv_layer(64, 128, stride=2)

        self.preres_block0 = self._make_preres_block(8)
        self.preres_block1 = self._make_preres_block(16)
        self.preres_block2 = self._make_preres_block(32)
        self.preres_block3 = self._make_preres_block(64)
        self.preres_block4 = self._make_preres_block(128)

        self.output = torch.nn.Sequential(
            torch.nn.Dropout(p=0.50),
            torch.nn.Linear(128, 1),
        )

        self.input_gradient = None
        self.gradients = None

        # # Freeze the encoder layers and add a classifier head
        # for param in self.parameters():
        #     param.requires_grad = False

        self.classifier_head = torch.nn.Sigmoid()

    def _make_conv_layer(self, in_planes, out_planes, stride=1):

        return torch.nn.Conv3d(
            in_planes,
            out_planes,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )

    def _make_preres_block(self, planes):

        return PreActivationResBlock(planes=planes)

    def forward(self, x0):

        if (not self.training) and (x0.requires_grad):
            h = x0.register_hook(self.input_gradient_hook)

        x0 = self.conv_layer0(x0)
        x1 = self.preres_block0(x0)
        x1 = x1.add(x0)

        x1 = self.conv_layer1(x1)
        x2 = self.preres_block1(x1)
        x2 = x2.add(x1)

        x2 = self.conv_layer2(x2)
        x3 = self.preres_block2(x2)
        x3 = x3.add(x2)

        x3 = self.conv_layer3(x3)
        x4 = self.preres_block3(x3)
        x4 = x4.add(x3)

        x4 = self.conv_layer4(x4)
        x5 = self.preres_block4(x4)
        if (not self.training) and (x5.requires_grad):
            # print(f"DeepPETEncoder: triggered gradient hook")
            h = x5.register_hook(self.activation_gradient_hook)
        # make a copy of tensor and store as activation maps
        self.activation_maps = x5.detach().clone()
        x5 = x5.add(x4)

        # global average pooling 3d
        x5 = x5.mean(dim=(-3, -2, -1))
        x5 = self.output(x5)

        # return x5
        return self.classifier_head(x5)


class _DenseLayer(torch.nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super().__init__()
        self.add_module("norm1", torch.nn.BatchNorm3d(num_input_features))
        self.add_module("relu1", torch.nn.ReLU(inplace=True))
        self.add_module(
            "conv1",
            torch.nn.Conv3d(
                num_input_features,
                bn_size * growth_rate,
                kernel_size=1,
                stride=1,
                bias=False,
            ),
        )
        self.add_module("norm2", torch.nn.BatchNorm3d(bn_size * growth_rate))
        self.add_module("relu2", torch.nn.ReLU(inplace=True))
        self.add_module(
            "conv2",
            torch.nn.Conv3d(
                bn_size * growth_rate,
                growth_rate,
                kernel_size=3,
                stride=1,
                padding=1,
                bias=False,
            ),
        )
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super().forward(x)
        if self.drop_rate > 0:
            new_features = torch.nn.functional.dropout(
                new_features, p=self.drop_rate, training=self.training
            )
        return torch.cat([x, new_features], 1)


class _DenseBlock(torch.nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super().__init__()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate
            )
            self.add_module(f"denselayer{(i + 1)}", layer)


class _Transition(torch.nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super().__init__()
        self.add_module("norm", torch.nn.BatchNorm3d(num_input_features))
        self.add_module("relu", torch.nn.ReLU(inplace=True))
        self.add_module(
            "conv",
            torch.nn.Conv3d(
                num_input_features,
                num_output_features,
                kernel_size=1,
                stride=1,
                bias=False,
            ),
        )
        self.add_module("pool", torch.nn.AvgPool3d(kernel_size=2, stride=2))


class DeepPETDenseNetClassifier(torch.nn.Module):
    """Densenet-BC model class
    Args:
        growth_rate (int) - how many filters to add each layer (k in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
    """

    def __init__(
        self,
        no_max_pool=False,
        growth_rate=16,
        block_config=(3, 3, 3, 3),
        bn_size=4,
        drop_rate=0.25,
    ):

        super().__init__()

        # First convolution
        self.features = [
            (
                "conv1",
                torch.nn.Conv3d(
                    1,
                    8,
                    kernel_size=5,
                    stride=2,
                    padding=2,
                    bias=False,
                ),
            ),
            ("norm1", torch.nn.BatchNorm3d(8)),
            ("relu1", torch.nn.ReLU(inplace=True)),
        ]
        if not no_max_pool:
            self.features.append(
                ("pool1", torch.nn.MaxPool3d(kernel_size=3, stride=2, padding=1))
            )
        self.features = torch.nn.Sequential(OrderedDict(self.features))

        # Each denseblock
        num_features = 8
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
            )
            self.features.add_module(f"denseblock{(i + 1)}", block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(
                    num_input_features=num_features,
                    num_output_features=num_features // 2,
                )
                self.features.add_module(f"transition{(i + 1)}", trans)
                num_features = num_features // 2

        # final batch norm
        self.features.add_module("norm5", torch.nn.BatchNorm3d(num_features))
        # final dense layer
        self.classifier = torch.nn.Linear(num_features, 1)

    def forward(self, x):
        features = self.features(x)
        out = torch.nn.functional.adaptive_avg_pool3d(features, output_size=(1, 1, 1)).view(
            features.size(0), -1
        )
        out = self.classifier(out)
        return out

# Predict A+

In [9]:
# Select PET_IMAGE and A+ from df
X = df['PET_IMAGE'].tolist()  # This will be a list of torch.Tensor objects
y = df['A+'].values     # This will be a numpy array of labels

print(f'X length: {len(X)}, PET image shape: {X[0].shape}, y shape: {y.shape}')

X length: 149, PET image shape: torch.Size([101, 116, 96]), y shape: (149,)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
print(f'Train X length: {len(X_train)}, Test X length: {len(X_test)} with shapes {X_train[0].shape}, {X_test[0].shape}')
print(f'Train y shape: {y_train.shape}, Test y shape: {y_test.shape}')

Train X length: 119, Test X length: 30 with shapes torch.Size([101, 116, 96]), torch.Size([101, 116, 96])
Train y shape: (119,), Test y shape: (30,)


In [11]:
# Save the train and test sets
torch.save(X_train, 'data/pet_csf/X_train_aplus.pt')
torch.save(y_train, 'data/pet_csf/y_train_aplus.pt')
torch.save(X_test, 'data/pet_csf/X_test_aplus.pt')
torch.save(y_test, 'data/pet_csf/y_test_aplus.pt')

In [25]:
X_train = torch.load('data/pet_csf/X_train_aplus.pt', weights_only=False)
X_test = torch.load('data/pet_csf/X_test_aplus.pt', weights_only=False)
y_train = torch.load('data/pet_csf/y_train_aplus.pt', weights_only=False)
y_test = torch.load('data/pet_csf/y_test_aplus.pt', weights_only=False)

In [26]:
batch_size = 2
model = DeepPETEncoderGradCAM()
model.load_state_dict(torch.load('weights/deeppet.pth', weights_only=True)['model'])
model.to(device)

DeepPETEncoderGradCAM(
  (conv_layer0): Conv3d(1, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
  (conv_layer1): Conv3d(8, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
  (conv_layer2): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
  (conv_layer3): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
  (conv_layer4): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
  (preres_block0): PreActivationResBlock(
    (conv1): Conv3d(8, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (conv2): Conv3d(8, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (bn1): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (dropo

Test the pretrained model before tuning

In [16]:
# Compute the accuracy on the train set
model.eval()
with torch.no_grad():
    train_dataset = PETDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    y_pred_train = []
    y_true_train = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        y_pred_train.extend(outputs.cpu().numpy())
        y_true_train.extend(labels.cpu().numpy())
    y_pred_train = (torch.tensor(
        y_pred_train) > 0.5).float().numpy()  # Convert probabilities to binary predictions
    y_true_train = torch.tensor(y_true_train).numpy()
    train_accuracy = accuracy_score(y_true_train, y_pred_train)
    train_f1 = f1_score(y_true_train, y_pred_train)
    train_roc_auc = roc_auc_score(y_true_train, y_pred_train)
    print(
        f'Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1:.4f}, Train ROC AUC: {train_roc_auc:.4f}')

Train Accuracy: 0.8151, Train F1 Score: 0.8830, Train ROC AUC: 0.6958


C:\Users\Dhruv Khanna\AppData\Local\Temp\ipykernel_19596\3392771040.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  y_pred_train = (torch.tensor(


In [17]:
# Compute the accuracy on the test set
with torch.no_grad():
    test_dataset = PETDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    y_pred_test = []
    y_true_test = []

    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        y_pred_test.extend(outputs.cpu().numpy())
        y_true_test.extend(labels.cpu().numpy())
    y_pred_test = (torch.tensor(
        y_pred_test) > 0.5).float().numpy()  # Convert probabilities to binary predictions
    y_true_test = torch.tensor(y_true_test).numpy()
    test_accuracy = accuracy_score(y_true_test, y_pred_test)
    test_f1 = f1_score(y_true_test, y_pred_test)
    test_roc_auc = roc_auc_score(y_true_test, y_pred_test)
    print(
        f'Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}')

Test Accuracy: 0.8667, Test F1 Score: 0.9231, Test ROC AUC: 0.6800


Tune the pretrained model

In [ ]:
inputs = torch.stack([torch.unsqueeze(img, 0) for img in X_train], dim=0)  # Add channel dimension
labels = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Convert labels to float and add a channel dimension
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses = []

batch_size = 2  # Adjust based on your GPU memory
train_dataset = PETDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
total_batches = len(train_loader)

for epoch in range(1, epochs + 1):
    model.train()
    batch_counter = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        batch_counter += 1
        print(f'Processing batch {batch_counter}/{total_batches} of epoch {epoch}')
        optimizer.zero_grad()
        outputs = model(inputs)
        # print(f'Output shape: {outputs.shape}, Labels shape: {labels.shape}')
        # print(f'Output: {outputs[:5]}, Labels: {labels[:5]}')
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        # Save the best model based on loss
        if loss.item() < min(train_losses, default=float('inf')):
            torch.save(model.state_dict(), 'weights/best_deeppet_csf_tuned_aplus.pth')
    del inputs, labels, outputs  # Clear variables to free memory
    torch.cuda.empty_cache()  # Clear GPU memory after each epoch
    gc.collect()  # Collect garbage to free up memory

    print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), 'weights/deeppet_csf_tuned_aplus.pth')

Processing batch 1/60 of epoch 1
Processing batch 2/60 of epoch 1
Processing batch 3/60 of epoch 1
Processing batch 4/60 of epoch 1
Processing batch 5/60 of epoch 1
Processing batch 6/60 of epoch 1
Processing batch 7/60 of epoch 1
Processing batch 8/60 of epoch 1
Processing batch 9/60 of epoch 1
Processing batch 10/60 of epoch 1
Processing batch 11/60 of epoch 1
Processing batch 12/60 of epoch 1
Processing batch 13/60 of epoch 1
Processing batch 14/60 of epoch 1
Processing batch 15/60 of epoch 1
Processing batch 16/60 of epoch 1
Processing batch 17/60 of epoch 1
Processing batch 18/60 of epoch 1
Processing batch 19/60 of epoch 1
Processing batch 20/60 of epoch 1
Processing batch 21/60 of epoch 1
Processing batch 22/60 of epoch 1
Processing batch 23/60 of epoch 1
Processing batch 24/60 of epoch 1
Processing batch 25/60 of epoch 1
Processing batch 26/60 of epoch 1
Processing batch 27/60 of epoch 1
Processing batch 28/60 of epoch 1
Processing batch 29/60 of epoch 1
Processing batch 30/60 

In [ ]:
# Compute the accuracy on the train set
model.eval()
with torch.no_grad():
    train_dataset = PETDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    y_pred_train = []
    y_true_train = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        y_pred_train.extend(outputs.cpu().numpy())
        y_true_train.extend(labels.cpu().numpy())
    y_pred_train = (torch.tensor(
        y_pred_train) > 0.5).float().numpy()  # Convert probabilities to binary predictions
    y_true_train = torch.tensor(y_true_train).numpy()
    train_accuracy = accuracy_score(y_true_train, y_pred_train)
    train_f1 = f1_score(y_true_train, y_pred_train)
    train_roc_auc = roc_auc_score(y_true_train, y_pred_train)
    print(
        f'Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1:.4f}, Train ROC AUC: {train_roc_auc:.4f}')

In [ ]:
# Compute the accuracy on the test set
with torch.no_grad():
    test_dataset = PETDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    y_pred_test = []
    y_true_test = []

    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        y_pred_test.extend(outputs.cpu().numpy())
        y_true_test.extend(labels.cpu().numpy())
    y_pred_test = (torch.tensor(
        y_pred_test) > 0.5).float().numpy()  # Convert probabilities to binary predictions
    y_true_test = torch.tensor(y_true_test).numpy()
    test_accuracy = accuracy_score(y_true_test, y_pred_test)
    test_f1 = f1_score(y_true_test, y_pred_test)
    test_roc_auc = roc_auc_score(y_true_test, y_pred_test)
    print(
        f'Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}')

# Predict T+

In [19]:
# Select PET_IMAGE and T+ from df
X = df['PET_IMAGE'].tolist()  # This will be a list of torch.Tensor objects
y = df['T+'].values     # This will be a numpy array of labels

print(f'X length: {len(X)}, PET image shape: {X[0].shape}, y shape: {y.shape}')

X length: 149, PET image shape: torch.Size([101, 116, 96]), y shape: (149,)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
print(f'Train X length: {len(X_train)}, Test X length: {len(X_test)} with shapes {X_train[0].shape}, {X_test[0].shape}')
print(f'Train y shape: {y_train.shape}, Test y shape: {y_test.shape}')

Train X length: 119, Test X length: 30 with shapes torch.Size([101, 116, 96]), torch.Size([101, 116, 96])
Train y shape: (119,), Test y shape: (30,)


In [21]:
# Save the train and test sets
torch.save(X_train, 'data/pet_csf/X_train_tplus.pt')
torch.save(y_train, 'data/pet_csf/y_train_tplus.pt')
torch.save(X_test, 'data/pet_csf/X_test_tplus.pt')
torch.save(y_test, 'data/pet_csf/y_test_tplus.pt')

In [ ]:
X_train = torch.load('data/pet_csf/X_train_tplus.pt', weights_only=False)
X_test = torch.load('data/pet_csf/X_test_tplus.pt', weights_only=False)
y_train = torch.load('data/pet_csf/y_train_tplus.pt', weights_only=False)
y_test = torch.load('data/pet_csf/y_test_tplus.pt', weights_only=False)

Test the pretrained model before tuning

In [ ]:
batch_size = 2
model = DeepPETEncoderGradCAM()
model.load_state_dict(torch.load('weights/deeppet.pth', weights_only=True)['model'])
model.to(device)

In [ ]:
# Compute the accuracy on the train set
model.eval()
with torch.no_grad():
    train_dataset = PETDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    y_pred_train = []
    y_true_train = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        y_pred_train.extend(outputs.cpu().numpy())
        y_true_train.extend(labels.cpu().numpy())
    y_pred_train = (torch.tensor(
        y_pred_train) > 0.5).float().numpy()  # Convert probabilities to binary predictions
    y_true_train = torch.tensor(y_true_train).numpy()
    train_accuracy = accuracy_score(y_true_train, y_pred_train)
    train_f1 = f1_score(y_true_train, y_pred_train)
    train_roc_auc = roc_auc_score(y_true_train, y_pred_train)
    print(
        f'Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1:.4f}, Train ROC AUC: {train_roc_auc:.4f}')

In [ ]:
# Compute the accuracy on the test set
with torch.no_grad():
    test_dataset = PETDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    y_pred_test = []
    y_true_test = []

    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        y_pred_test.extend(outputs.cpu().numpy())
        y_true_test.extend(labels.cpu().numpy())
    y_pred_test = (torch.tensor(
        y_pred_test) > 0.5).float().numpy()  # Convert probabilities to binary predictions
    y_true_test = torch.tensor(y_true_test).numpy()
    test_accuracy = accuracy_score(y_true_test, y_pred_test)
    test_f1 = f1_score(y_true_test, y_pred_test)
    test_roc_auc = roc_auc_score(y_true_test, y_pred_test)
    print(
        f'Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}')

Tune the pretrained model

In [ ]:
inputs = torch.stack([torch.unsqueeze(img, 0) for img in X_train], dim=0)  # Add channel dimension
labels = torch.tensor(y_train, dtype=torch.float32).unsqueeze(
    1)  # Convert labels to float and add a channel dimension
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses = []

batch_size = 2  # Adjust based on your GPU memory
train_dataset = PETDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
total_batches = len(train_loader)

for epoch in range(1, epochs + 1):
    model.train()
    batch_counter = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        batch_counter += 1
        print(f'Processing batch {batch_counter}/{total_batches} of epoch {epoch}')
        optimizer.zero_grad()
        outputs = model(inputs)
        # print(f'Output shape: {outputs.shape}, Labels shape: {labels.shape}')
        # print(f'Output: {outputs[:5]}, Labels: {labels[:5]}')
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        # Save the best model based on loss
        if loss.item() < min(train_losses, default=float('inf')):
            torch.save(model.state_dict(), 'weights/best_deeppet_csf_tuned_tplus.pth')
    del inputs, labels, outputs  # Clear variables to free memory
    torch.cuda.empty_cache()  # Clear GPU memory after each epoch
    gc.collect()  # Collect garbage to free up memory

    print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), 'weights/deeppet_csf_tuned_tplus.pth')

In [ ]:
# Compute the accuracy on the train set
model.eval()
with torch.no_grad():
    train_dataset = PETDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    y_pred_train = []
    y_true_train = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        y_pred_train.extend(outputs.cpu().numpy())
        y_true_train.extend(labels.cpu().numpy())
    y_pred_train = (torch.tensor(
        y_pred_train) > 0.5).float().numpy()  # Convert probabilities to binary predictions
    y_true_train = torch.tensor(y_true_train).numpy()
    train_accuracy = accuracy_score(y_true_train, y_pred_train)
    train_f1 = f1_score(y_true_train, y_pred_train)
    train_roc_auc = roc_auc_score(y_true_train, y_pred_train)
    print(
        f'Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1:.4f}, Train ROC AUC: {train_roc_auc:.4f}')

In [ ]:
# Compute the accuracy on the test set
with torch.no_grad():
    test_dataset = PETDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    y_pred_test = []
    y_true_test = []

    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        y_pred_test.extend(outputs.cpu().numpy())
        y_true_test.extend(labels.cpu().numpy())
    y_pred_test = (torch.tensor(
        y_pred_test) > 0.5).float().numpy()  # Convert probabilities to binary predictions
    y_true_test = torch.tensor(y_true_test).numpy()
    test_accuracy = accuracy_score(y_true_test, y_pred_test)
    test_f1 = f1_score(y_true_test, y_pred_test)
    test_roc_auc = roc_auc_score(y_true_test, y_pred_test)
    print(
        f'Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}')

# Predict N+

In [22]:
# Select PET_IMAGE and T+ from df
X = df['PET_IMAGE'].tolist()  # This will be a list of torch.Tensor objects
y = df['T+'].values     # This will be a numpy array of labels

print(f'X length: {len(X)}, PET image shape: {X[0].shape}, y shape: {y.shape}')

X length: 149, PET image shape: torch.Size([101, 116, 96]), y shape: (149,)


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
print(f'Train X length: {len(X_train)}, Test X length: {len(X_test)} with shapes {X_train[0].shape}, {X_test[0].shape}')
print(f'Train y shape: {y_train.shape}, Test y shape: {y_test.shape}')

Train X length: 119, Test X length: 30 with shapes torch.Size([101, 116, 96]), torch.Size([101, 116, 96])
Train y shape: (119,), Test y shape: (30,)


In [24]:
# Save the train and test sets
torch.save(X_train, 'data/pet_csf/X_train_nplus.pt')
torch.save(y_train, 'data/pet_csf/y_train_nplus.pt')
torch.save(X_test, 'data/pet_csf/X_test_nplus.pt')
torch.save(y_test, 'data/pet_csf/y_test_nplus.pt')

In [ ]:
X_train = torch.load('data/pet_csf/X_train_nplus.pt', weights_only=False)
X_test = torch.load('data/pet_csf/X_test_nplus.pt', weights_only=False)
y_train = torch.load('data/pet_csf/y_train_nplus.pt', weights_only=False)
y_test = torch.load('data/pet_csf/y_test_nplus.pt', weights_only=False)

Test the pretrained model before tuning

In [ ]:
batch_size = 2
model = DeepPETEncoderGradCAM()
model.load_state_dict(torch.load('weights/deeppet.pth', weights_only=True)['model'])
model.to(device)

In [ ]:
# Compute the accuracy on the train set
model.eval()
with torch.no_grad():
    train_dataset = PETDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    y_pred_train = []
    y_true_train = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        y_pred_train.extend(outputs.cpu().numpy())
        y_true_train.extend(labels.cpu().numpy())
    y_pred_train = (torch.tensor(
        y_pred_train) > 0.5).float().numpy()  # Convert probabilities to binary predictions
    y_true_train = torch.tensor(y_true_train).numpy()
    train_accuracy = accuracy_score(y_true_train, y_pred_train)
    train_f1 = f1_score(y_true_train, y_pred_train)
    train_roc_auc = roc_auc_score(y_true_train, y_pred_train)
    print(
        f'Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1:.4f}, Train ROC AUC: {train_roc_auc:.4f}')

In [ ]:
# Compute the accuracy on the test set
with torch.no_grad():
    test_dataset = PETDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    y_pred_test = []
    y_true_test = []

    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        y_pred_test.extend(outputs.cpu().numpy())
        y_true_test.extend(labels.cpu().numpy())
    y_pred_test = (torch.tensor(
        y_pred_test) > 0.5).float().numpy()  # Convert probabilities to binary predictions
    y_true_test = torch.tensor(y_true_test).numpy()
    test_accuracy = accuracy_score(y_true_test, y_pred_test)
    test_f1 = f1_score(y_true_test, y_pred_test)
    test_roc_auc = roc_auc_score(y_true_test, y_pred_test)
    print(
        f'Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}')

Tune the pretrained model

In [ ]:
inputs = torch.stack([torch.unsqueeze(img, 0) for img in X_train], dim=0)  # Add channel dimension
labels = torch.tensor(y_train, dtype=torch.float32).unsqueeze(
    1)  # Convert labels to float and add a channel dimension
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses = []

batch_size = 2  # Adjust based on your GPU memory
train_dataset = PETDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
total_batches = len(train_loader)

for epoch in range(1, epochs + 1):
    model.train()
    batch_counter = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        batch_counter += 1
        print(f'Processing batch {batch_counter}/{total_batches} of epoch {epoch}')
        optimizer.zero_grad()
        outputs = model(inputs)
        # print(f'Output shape: {outputs.shape}, Labels shape: {labels.shape}')
        # print(f'Output: {outputs[:5]}, Labels: {labels[:5]}')
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        # Save the best model based on loss
        if loss.item() < min(train_losses, default=float('inf')):
            torch.save(model.state_dict(), 'weights/best_deeppet_csf_tuned_nplus.pth')
    del inputs, labels, outputs  # Clear variables to free memory
    torch.cuda.empty_cache()  # Clear GPU memory after each epoch
    gc.collect()  # Collect garbage to free up memory

    print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), 'weights/deeppet_csf_tuned_nplus.pth')

In [ ]:
# Compute the accuracy on the train set
model.eval()
with torch.no_grad():
    train_dataset = PETDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    y_pred_train = []
    y_true_train = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        y_pred_train.extend(outputs.cpu().numpy())
        y_true_train.extend(labels.cpu().numpy())
    y_pred_train = (torch.tensor(
        y_pred_train) > 0.5).float().numpy()  # Convert probabilities to binary predictions
    y_true_train = torch.tensor(y_true_train).numpy()
    train_accuracy = accuracy_score(y_true_train, y_pred_train)
    train_f1 = f1_score(y_true_train, y_pred_train)
    train_roc_auc = roc_auc_score(y_true_train, y_pred_train)
    print(
        f'Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1:.4f}, Train ROC AUC: {train_roc_auc:.4f}')

In [ ]:
# Compute the accuracy on the test set
with torch.no_grad():
    test_dataset = PETDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    y_pred_test = []
    y_true_test = []

    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        y_pred_test.extend(outputs.cpu().numpy())
        y_true_test.extend(labels.cpu().numpy())
    y_pred_test = (torch.tensor(
        y_pred_test) > 0.5).float().numpy()  # Convert probabilities to binary predictions
    y_true_test = torch.tensor(y_true_test).numpy()
    test_accuracy = accuracy_score(y_true_test, y_pred_test)
    test_f1 = f1_score(y_true_test, y_pred_test)
    test_roc_auc = roc_auc_score(y_true_test, y_pred_test)
    print(
        f'Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}')